# Xing Xiang's Upright vs Rotated classifer for Homework

The aim is to classify between homework images that are upright and rotated, either 90 degrees cw/ccw.


In [1]:
# First, I construct the model architecture and make use of ResNet50, and leave only the last layer trainable.
# Number of classes is 2. Either upright or rotated.


from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

import tensorflow as tf

tf.keras.backend.clear_session()

num_classes = 2

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg'))
my_new_model.add(Dense(256, activation='relu'))
my_new_model.add(Dropout(rate=0.5))
my_new_model.add(Dense(64, activation='relu'))
my_new_model.add(Dense(num_classes, activation='softmax'))


# Indicate whether the first layer should be trained/changed or not.
my_new_model.layers[0].trainable = False

my_new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 24,128,834
Trainable params: 541,122
Non-trainable params: 23,587,712
_________________________________________________________________


In [2]:
from tensorflow.keras.optimizers import SGD

my_new_model.compile(optimizer=SGD(learning_rate=0.003, momentum=0.9, nesterov=True, name='SGD'), 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

In [3]:
# use image generators to load my images sourced from google images. I added data augmentation for the train generator only 
# to increase the number of train data.


from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 100
data_generator = ImageDataGenerator(preprocess_input,  width_shift_range=0.4,
    height_shift_range=0.4, zoom_range=0.4, horizontal_flip=True)

# normal_generator = ImageDataGenerator(preprocess_input)

train_generator = data_generator.flow_from_directory(
                                        directory="./train",
                                        target_size=(image_size, image_size),
                                        batch_size=100,
                                        class_mode='categorical',
                                        shuffle=True)

validation_generator = data_generator.flow_from_directory(
                                        directory="./test",
                                        target_size=(image_size, image_size),
                                        batch_size=10,
                                        class_mode='categorical',
                                        shuffle=True)




Found 1050 images belonging to 2 classes.
Found 144 images belonging to 2 classes.


In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# make use of callbacks to stop early upon sign of overtraining, and also to save weights


checkpoint_callback = ModelCheckpoint(
    filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='max',
    save_best_only=True)

earlystopping_callback = EarlyStopping(monitor='val_loss', patience=0)

In [5]:
# Using training data to fit my model and validate against validation data

fit_stats = my_new_model.fit(x=train_generator, validation_data=validation_generator, steps_per_epoch=35, validation_steps=25 ,epochs=25, verbose=1, callbacks=[earlystopping_callback,checkpoint_callback])

C:\Users\xx\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/25
35/35 [==============================] - 12s 340ms/step - loss: 0.5486 - accuracy: 0.7146 - val_loss: 0.3987 - val_accuracy: 0.7828
Epoch 2/25
35/35 [==============================] - 10s 280ms/step - loss: 0.3688 - accuracy: 0.8391 - val_loss: 0.2846 - val_accuracy: 0.8487
Epoch 3/25
35/35 [==============================] - 10s 286ms/step - loss: 0.2971 - accuracy: 0.8773 - val_loss: 0.3077 - val_accuracy: 0.8782


In [6]:
fit_stats.history

{'loss': [0.5485996603965759, 0.36879879236221313, 0.29712092876434326],
 'accuracy': [0.7146268486976624, 0.8391044735908508, 0.8773134350776672],
 'val_loss': [0.39867883920669556, 0.2846055030822754, 0.3077290952205658],
 'val_accuracy': [0.7827869057655334, 0.848739504814148, 0.8781512379646301]}

In [7]:
# Define function to prepare test images easily

from PIL import Image
import numpy as np

def prepare_img(filePath):
    out = np.array(Image.open(filePath))
    out = out.reshape((-1, 100, 100, 3))
    return out

In [8]:
# Test out model with a folder of test images

from pathlib import Path

# Getting class labels
labels = (validation_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
# print(labels)

num_correct = 0
num_total = 0
for filePath in Path(r"C:\Users\xx\Workspace\Deep learning tryouts\test").glob("**/*.jpg"):
    num_total += 1
    
    # use model to predict class
    pred = my_new_model.predict(prepare_img(filePath))
    
    predicted_idx = np.argmax(pred,axis=1)[0]
    predicted_class = labels[predicted_idx]
#     print(f"Label-{filePath.parent.stem}, predicted-{predicted_class}") 
    
    if filePath.parent.stem == predicted_class:
        num_correct += 1
        
    
accuracy = round(num_correct / num_total, 2) 
print(f"{num_correct}/{num_total} --> Accuracy: {accuracy}")



133/144 --> Accuracy: 0.92
